# ZRP User Guide
The purpose of this notebook is to illustrate how to use ZRP, the main class of the zrp package that processes user input data &  returns race/ethnicity predictions

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser
import pandas as pd
import sys
import os
import re
import warnings

## Set source code path here

In [3]:
warnings.filterwarnings(action='once')
home = expanduser('~')

src_path = '{}/zest/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp import ZRP
from zrp.prepare.utils import load_file

/usr/local/lib/python3.9/site-packages/category_encoders/utils.py:5: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  version = LooseVersion(pd.__version__)
/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Load sample data for prediction
Load list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [5]:
nj_mayors = load_file(src_path + "/examples/2022-nj-mayors.csv")
nj_mayors.shape

(565, 18)

In [6]:
nj_mayors

,MUNI CODE,MUNI NAME,COUNTY,ADDRESS 1,ADDRESS 2,CITY,STATE,ZIP,PHONE,FAX,MAYOR NAME,TERM START,TERM END,FORM,TERM LEGNTH,EMAIL,SOCIAL MEDIA HANDLE,Municipal Contact List
0,1330,Aberdeen Township,Monmouth,One Aberdeen Square,NaN,Aberdeen,NJ,07747-2300,(732) 583-4200,NaN,Fred Tagliarini,NaN,12/31/2025,COUNCIL-MANAGER,4,fred.tagliarini@aberdeennj.org,NaN,NaN
1,0101,Absecon City,Atlantic,Absecon Municipal Complex,500 Mill Road,Absecon,NJ,08201,(609) 641-0663,(609) 645-5098,Kimberly Horton,NaN,12/31/2024,MAYOR-COUNCIL,3,khorton@abseconnj.org,NaN,NaN
2,1001,Alexandria Township,Hunterdon,782 Frenchtown Road,NaN,Milford,NJ,08848,(908) 996-7071,NaN,Gabe Plumer,NaN,12/31/2022,TOWNSHIP,3,clerk@alexandrianj.gov,NaN,NaN
3,2101,Allamuchy Township,Warren,Post Office Box A,NaN,Allamuchy,NJ,07820,(908) 852-5132,NaN,Rosemary Tuohy,NaN,12/31/2024,FAULKNER ACT,3,mayor@allamuchynj.org,NaN,NaN
4,0201,Allendale Borough,Bergen,500 West Crescent Avenue,NaN,Allendale,NJ,07401,(201) 818-4400,NaN,Ari Bernstein,NaN,12/31/2022,NaN,NaN,aribernstein@allendalenj.gov,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0269,Wood-Ridge Borough,Bergen,85 Humboldt Street,NaN,Wood-Ridge,NJ,07075-2344,(201) 939-0202,NaN,Paul A Sarlo,NaN,12/31/2023,NaN,NaN,psarlo@njwoodridge.org,NaN,NaN
561,1715,Woodstown Borough,Salem,Post Office Box 286,NaN,Woodstown,NJ,08098,(856) 769-2200,NaN,Donald Dietrich,NaN,12/31/2023,NaN,NaN,Don.dietrich@comcast.net,NaN,NaN
562,0824,Woolwich Township,Gloucester,120 Village Green Drive,NaN,Woolwich Township,NJ,08085-3180,(856) 467-2666,NaN,Craig Frederick,NaN,12/31/2024,NaN,NaN,cfrederick@woolwichtwp.org,NaN,NaN
563,0340,Wrightstown Borough,Burlington,21 Saylors Pond Road,NaN,Wrightstown,NJ,08562,(609) 723-4450,(609) 723-7137,Donald Cottrell,NaN,12/31/2022,NaN,NaN,mayor@wrightstownborough.com,NaN,NaN


### Wrangle NJ mayor data for predictions
This parsing of the NJ mayors file will leave some NA's, but it is sufficient for demonstration purposes


In [7]:
zrp_sample = pd.DataFrame(columns=['first_name', 'middle_name', 'last_name', 'house_number', 'street_address', 'city', 'state', 'zip_code'])

Prepare Names

In [13]:
nj_mayors['MAYOR NAME'].str.replace(' Jr\\.$', '', regex=True)
nj_mayors['MAYOR NAME'].str.replace(' Jr$', '', regex=True)
split_mayor_names = nj_mayors['MAYOR NAME'].str.split(' ')
zrp_sample['first_name'] = split_mayor_names.str[0]
zrp_sample['last_name'] = split_mayor_names.str[-1]

City, State, Zip

In [14]:
zrp_sample['city'] = nj_mayors['CITY']
zrp_sample['state'] = nj_mayors['STATE']
zrp_sample['zip_code'] = nj_mayors['ZIP']

Address

In [15]:
zrp_sample['house_number'] = nj_mayors['ADDRESS 1'].str.extract('([0-9]+)')
zrp_sample['street_address'] = nj_mayors['ADDRESS 1'].str.extract('.*[0-9]+([^0-9]+)')


In [30]:
zrp_sample['ZEST_KEY'] = zrp_sample.index.astype(str)  #must specify key to establish correspondence between inputs and outputs
zrp_sample

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Fred,NaN,Tagliarini,NaN,NaN,Aberdeen,NJ,07747-2300,0
1,Kimberly,NaN,Horton,NaN,NaN,Absecon,NJ,08201,1
2,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
3,Rosemary,NaN,Tuohy,NaN,NaN,Allamuchy,NJ,07820,3
4,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
...,...,...,...,...,...,...,...,...,...
560,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
561,Donald,NaN,Dietrich,286,NaN,Woodstown,NJ,08098,561
562,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562
563,Donald,NaN,Cottrell,21,Saylors Pond Road,Wrightstown,NJ,08562,563


In [31]:
%%time
zest_race_predictor = ZRP()
zest_race_predictor.fit()
output = zest_race_predictor.transform(zrp_sample)

Directory already exists
Data is loaded
   Formatting P1
   Formatting P2
reduce whitespace

[Start] Preparing geo data
  The following states are included in the data: ['NJ']
   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
/Users/j/zest/zrp/zrp/prepare/../data/processed
      ...address cleaning


  0%|                                                   | 0/565 [00:00<?, ?it/s][Parallel(n_jobs=49)]: Using backend ThreadingBackend with 49 concurrent workers.
[Parallel(n_jobs=49)]: Done 102 tasks      | elapsed:    0.0s
[Parallel(n_jobs=49)]: Done 352 tasks      | elapsed:    0.0s
100%|██████████████████████████████████████| 565/565 [00:00<00:00, 12111.42it/s]

      ...replicating address
         ...Base
         ...Map street suffixes...



[Parallel(n_jobs=49)]: Done 565 out of 565 | elapsed:    0.1s finished
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1459b05e0>


         ...Mapped & split by street suffixes...
         ...Number processing...

     Address dataframe expansion is complete! (n=1010)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping
Directory already exists
Output saved
   [Completed] Mapping geo data
[Completed] Preparing geo data

[Start] Preparing ACS data
   ...loading ACS lookup tables
   ... combining ACS & user input data
ZEST_KEY
 ...Copy dataframes
 ...Block group
 ...Census tract
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete
[Complete] Preparing ACS data


/Users/j/zest/zrp/zrp/modeling/predict.py:124: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  identifiedRaces = subset.idxmax(axis=1)


Output saved


/Users/j/zest/zrp/zrp/modeling/models/block_group/src/app_preprocessor.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_result = non_compound.append(compound_result).reset_index(drop=True)
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2739.58it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
/Users/j/zest/zrp/zrp/modeling/models/block_group/src/app_preprocessor.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_result = non_compound.append(compound_result).reset_index(drop=True)
/Users/j/zest/zrp/zrp/modeling/models/block_group/src/acs_scaler.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat

Directory already exists
Output saved
CPU times: user 35.9 s, sys: 5.08 s, total: 41 s
Wall time: 36.9 s


/Users/j/zest/zrp/zrp/modeling/predict.py:124: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  identifiedRaces = subset.idxmax(axis=1)


In [32]:
output

,AAPI,AIAN,BLACK,HISPANIC,WHITE,source_block_group,source_zip_code,OTHER,source_bisg
270,0.003071,0.113848,0.003382,0.006237,0.873462,1.0,NaN,NaN,NaN
302,0.063315,0.000668,0.016008,0.065184,0.854824,1.0,NaN,NaN,NaN
356,0.000608,0.000292,0.000695,0.001195,0.997210,1.0,NaN,NaN,NaN
359,0.000897,0.000425,0.002105,0.038322,0.958252,1.0,NaN,NaN,NaN
1,0.034340,0.016307,0.268386,0.023059,0.657908,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
88,0.038122,0.000537,0.002971,0.027246,0.931124,NaN,1.0,NaN,NaN
91,0.039716,0.007949,0.034505,0.161497,0.756334,NaN,1.0,NaN,NaN
94,0.024983,0.008647,0.001998,0.004494,0.959878,NaN,1.0,NaN,NaN
97,0.008535,0.000498,0.044080,0.008873,0.938014,NaN,1.0,NaN,NaN
